In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-h348hvoy
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-h348hvoy
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=8095d7556677c802540cccc787abf573b95021efd8a78cf8fbde2ba98deb5bbc
  Stored in directory: /tmp/pip-ephem-wheel-cache-2p1bu02i/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
/**
 * --------------------------------------------------------
 * Universidad del Valle de Guatemala
 * CC3056 - Programación de Microprocesadores
 * --------------------------------------------------------
 * HC-SR04.ipynb
 * Proyecto 3
 * --------------------------------------------------------
 * Javier Prado -21486
 * Angel Perez - 21298
 * Bryan Espana - 21550
 * --------------------------------------------------------
 */

#include <stdio.h>

#include <cuda_runtime.h>

#include <fstream>

#include <string>

#include <iostream>

#include <vector>

#Instruccion que usaran los hilos Subrutina llamada desde el Host y ejecutado en Device.
__global__ void get_time(float * D, float * T, int N) {
  int i = blockDim.x * blockIdx.x + threadIdx.x;
  if (i < N) {      
    T[i] = D[i] / 0.0343;
  }
}

std::vector < float > get_txt_data(std::string path) {#funcion para obtener los datos del txt
  std::vector < float > data;

  std::ifstream file(path);#ruta del archivo
  std::string str;

  int first = 1;#eliminamos el encabezado
  while (std::getline(file, str)) {
    if (first == 0) {
      data.push_back(std::stof(str));#colocamos y convertimos en float los datos de string
    }
    first = 0;
  }
  return data;
}

int main() {
  std::vector < float > read_data = get_txt_data("Datos1.txt");#funcion para sacar los datos de datos1.txt

  int numElements = read_data.size(); #numeor de elementos --> datos
  size_t size = numElements * sizeof(float);

  #asignamos la entrada de host en su vector de tiempo y distance
  float * h_D = (float * ) malloc(size);
  float * h_T = (float * ) malloc(size);

  #inicializamos el vector de datos en host
  for (int i = 0; i < numElements; i++) {
    h_D[i] = read_data[i];
  }
  #asignamos el device a sus vectores
  float * d_D = NULL;
  cudaMalloc((void **)&d_D, size);   
  float * d_T = NULL
  ;cudaMalloc((void **)&d_T, size);   

  #Transferimos datos de host to device
  cudaMemcpy(d_D, h_D, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_T, h_T, size, cudaMemcpyHostToDevice);

  #operacion par ahilos y bloques
  int threadsPerBlock = 128; 
  int blocksPerGrid = (numElements + threadsPerBlock - 1) / threadsPerBlock;

  #ejecutamos kernel
  get_time<<<blocksPerGrid, threadsPerBlock>>>(d_D, d_T, numElements);
  #transferumis resultados de device a host
  cudaMemcpy(h_D, d_D, size, cudaMemcpyDeviceToHost);
  cudaMemcpy(h_T, d_T, size, cudaMemcpyDeviceToHost);

  #imprimimos resultados por cada distancia brindamos su tiempo
  printf("RESULTADOS:\n");
  for (int i = 0; i < numElements; i++) {
    printf("(Distance,Time)[%i]= (%.3f, %.5f)\n", i, h_D[i], h_T[i]);
  }
  
  #liberacion de memoria device disancia y time
  cudaFree(d_D);
  cudaFree(d_T);
  #Libreacion de memoria host disancia y time
  free(h_D);
  free(h_T);

  return 0;
}

RESULTADOS:
(Distance,Time)[0]= (4.010, 116.90963)
(Distance,Time)[1]= (3.280, 95.62682)
(Distance,Time)[2]= (3.050, 88.92128)
(Distance,Time)[3]= (3.040, 88.62974)
(Distance,Time)[4]= (2.740, 79.88338)
(Distance,Time)[5]= (2.920, 85.13120)
(Distance,Time)[6]= (2.740, 79.88338)
(Distance,Time)[7]= (2.980, 86.88047)
(Distance,Time)[8]= (2.440, 71.13703)
(Distance,Time)[9]= (2.760, 80.46647)
(Distance,Time)[10]= (3.170, 92.41983)
(Distance,Time)[11]= (4.180, 121.86588)
(Distance,Time)[12]= (4.850, 141.39941)
(Distance,Time)[13]= (5.810, 169.38776)
(Distance,Time)[14]= (6.590, 192.12828)
(Distance,Time)[15]= (7.200, 209.91254)
(Distance,Time)[16]= (8.010, 233.52771)
(Distance,Time)[17]= (9.110, 265.59766)
(Distance,Time)[18]= (9.760, 284.54813)
(Distance,Time)[19]= (11.040, 321.86588)

